In [ ]:
import sqlite3

import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import numpy as np
import pandas as pd
import seaborn as sns
from dask.distributed import Client
from matplotlib import pyplot as plt
from utils.constants import BigMarkets, SmallMarkets, Correspondence

In [ ]:
client = Client(n_workers=6)
client

In [ ]:
# PART 1 : Global analysis of sport videos and NBA videos available on the Youniverse dataset

In [ ]:
## What is the evolution of the number of videos on Youtube within the study period ?

In [ ]:
##  How did popularity of basketball evolve over time ?

In [ ]:
video_metadata_df = dd.read_parquet("./data/video_metadata/parquet")
video_metadata_df["upload_date"] = dd.to_datetime(video_metadata_df.upload_date)

In [ ]:
video_metadata_df.head()

In [ ]:
date_obj = video_metadata_df.upload_date.dt
group_by = [date_obj.year, date_obj.month, video_metadata_df.categories]

video_evolution = video_metadata_df[["upload_date"]].groupby(by=group_by).count().persist()
video_evolution_pd = video_evolution.compute()

In [ ]:
video_evolution_pd.sort_index(inplace=True)
video_evolution_pd = video_evolution_pd[video_evolution.index != 2019]

video_evolution_pd.index.names = ["year", "month", "categories"]
video_evolution_pd.reset_index(inplace=True)
# video_evolution_pd = video_evolution_pd[(video_evolution_pd["year"] != 2019)]  # TODO : fix this to also include first months of 2019
video_evolution_pd["date"] = video_evolution_pd["year"].astype("str") + "/" + video_evolution_pd["month"].astype("str")

In [ ]:
def plot_comparsion(df):
    fig = plt.figure(figsize=(15, 8))
    ax = fig.add_subplot(111)
    sns.lineplot(x="date", y="upload_date", hue="categories", data=df)
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start, end, 12))
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
video_evolution_pd_sport = video_evolution_pd[video_evolution_pd["categories"].isin(["Sports", "People & Blogs", "Entertainment"])]

In [ ]:
plot_comparsion(video_evolution_pd)

In [ ]:
plot_comparsion(video_evolution_pd_sport)

In [ ]:
video_metadata_df.count().compute()

In [ ]:
video_metadata_df_sports = video_metadata_df[video_metadata_df["categories"].isin(["Sports", "People & Blogs", "Entertainment"])]
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].str.lower()
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].apply(lambda x: f",{x},")

In [ ]:
# video_metadata_df_sports = video_metadata_df_sports[video_metadata_df_sports['tags'].str.contains('basketball')]


date_obj = video_metadata_df_sports.upload_date.dt
group_by = [date_obj.year, date_obj.month, video_metadata_df.categories]

video_metadata_df_sports_grouped = video_metadata_df_sports[["upload_date"]].groupby(by=group_by).count().persist()

In [ ]:
video_metadata_df_sports.head()